# Clark County Only, Crash Severity as Target

## Imports and Setup

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [3]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_6154/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


## Preprocess Data

In [4]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','injury_type','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured','property_damage_only' ], inplace = True)

In [5]:
# Filter for Clark County
traffic_df = traffic_df.loc[traffic_df['county'] == 'CLARK']
traffic_df.drop(columns = ['county'], inplace = True)

In [6]:
traffic_df.columns

Index(['x', 'y', 'crash_severity', 'crash_date', 'crash_year', 'crash_time',
       'weather', 'crash_type', 'total_vehicles', 'v1_type', 'v1_driver_age',
       'v1_action', 'v1_driver_factors', 'v1_driver_distracted',
       'v1_vehicle_factors', 'v1_most_harmful_event', 'v1_all_events',
       'v2_type', 'v2_driver_age', 'v2_action', 'v2_driver_factors',
       'v2_driver_distracted', 'v2_vehicle_factors', 'v2_most_harmful_event',
       'v2_all_events', 'nonmotorist_factors', 'factors_roadway', 'lighting',
       'hwy_factors', 'pedalcyclist', 'pedestrian', 'motorcyclist',
       'animaltype'],
      dtype='object')

In [7]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

# traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors','hwy_factors',], inplace = True)

In [8]:
traffic_df['crash_severity'].value_counts()

PROPERTY DAMAGE ONLY    91631
INJURY CRASH            77401
FATAL CRASH               788
Name: crash_severity, dtype: int64

In [9]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'PROPERTY DAMAGE ONLY': 0,
    'INJURY CRASH': 1,
    'FATAL CRASH': 2,    
}

traffic_df['crash_severity'] = traffic_df['crash_severity'].apply(lambda x: injury_num[x])

traffic_df['crash_severity'].value_counts()

0    91631
1    77401
2      788
Name: crash_severity, dtype: int64

In [10]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [11]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [12]:
traffic_df.dtypes

x                    float64
y                    float64
crash_severity         int64
crash_year             int64
weather               object
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 62, dtype: object

In [13]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))

In [14]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_severity,crash_year,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER ENVIRONMENTAL,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,0,2016,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,0,2016,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,0,2016,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0
5,-115.278964,36.136313,2,2016,1,35,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0
6,-115.162277,36.232014,1,2016,1,43,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0


## Model for Clark County with Crash Severity as Target and All Other Columns as Features

In [15]:
# Separate data into features and target
y = traffic_df_encoded['crash_severity'].values
X = traffic_df_encoded.drop(columns = 'crash_severity')

In [16]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(127365, 1231)

In [17]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,0,1
1,0,0
2,1,1
3,1,0
4,0,0


In [20]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.741161229537157

In [21]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Property Damage Only', 
                'Actual: Injury Crash',  
                'Actual: Fatal Crash']

column_values = ['Predicted: Property Damage Only', 
                'Predicted: Injury Crash', 
                'Predicted: Fatal Crash']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,17146,5761,1
Actual: Injury Crash,5029,14314,7
Actual: Fatal Crash,37,154,6


## Results for Clark County with Crash Severity as Target and All Other Columns as Features

In [22]:
# Display results
print('Clark County Data with Crash Severity as Target, All Other Columns as Features')
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,17146,5761,1
Actual: Injury Crash,5029,14314,7
Actual: Fatal Crash,37,154,6



Accuracy Score: 0.741161229537157

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.75      0.76     22908
           1       0.71      0.74      0.72     19350
           2       0.43      0.03      0.06       197

    accuracy                           0.74     42455
   macro avg       0.64      0.51      0.51     42455
weighted avg       0.74      0.74      0.74     42455



### Importances for Clark County with Crash Severity as Target and All Other Columns as Features

In [23]:
# Calculate importance of features
importances = rf_model.feature_importances_
sorted(zip(importances, X.columns), reverse = True)


[(0.05172966693523143, 'v1_driver_age'),
 (0.05095710793426273, 'y'),
 (0.050217670686244185, 'x'),
 (0.047893044549699844, 'v2_driver_age'),
 (0.04228479021564723, 'crash_day'),
 (0.039753380345746055, 'crash_hour'),
 (0.035882579384285775, 'crash_month'),
 (0.03105209353869472, 'crash_day_of_week'),
 (0.027714620423751665, 'factors_roadway_DRY'),
 (0.02588198703822462, 'factors_roadway_UNKNOWN'),
 (0.02525114519379698, 'lighting_UNKNOWN'),
 (0.020767994426487763, 'crash_year'),
 (0.02074712086277525, 'hwy_factors_1_UNKNOWN'),
 (0.01595396131619971, 'hwy_factors_1_NONE'),
 (0.014864918928796727, 'lighting_DAYLIGHT'),
 (0.014621554526123803, 'total_vehicles'),
 (0.010578686599815253, 'v1_vehicle_factors_1_FAILED TO YIELD RIGHT OF WAY'),
 (0.010454746567674087, 'v1_type_SEDAN, 4 DOOR'),
 (0.01022460672448796, 'crash_type_ANGLE'),
 (0.009788067948168343, 'crash_type_SIDESWIPE, OVERTAKING'),
 (0.008274754845318734, 'v2_type_SEDAN, 4 DOOR'),
 (0.008056576074434343, 'v1_action_GOING STRAIGH